<a href="https://colab.research.google.com/github/Zilleplus/MachineLearning/blob/feature%2Fadd-pytorch-examples/PyTorchQuickStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
print(type(training_data))
print(type(training_data))

<class 'torchvision.datasets.mnist.FashionMNIST'>
<class 'torchvision.datasets.mnist.FashionMNIST'>


In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(type(X))
    print("Shape of X [N, C, H, W]: ", X.shape)
    print(type(y))
    print("Shape of y: ", y.shape, y.dtype)
    break # just take one sample

<class 'torch.Tensor'>
Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
<class 'torch.Tensor'>
Shape of y:  torch.Size([64]) torch.int64


In [ ]:
print(type(test_dataloader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.310589  [    0/60000]
loss: 2.298421  [ 6400/60000]
loss: 2.279651  [12800/60000]
loss: 2.263791  [19200/60000]
loss: 2.247264  [25600/60000]
loss: 2.214040  [32000/60000]
loss: 2.216414  [38400/60000]
loss: 2.183689  [44800/60000]
loss: 2.185591  [51200/60000]
loss: 2.140024  [57600/60000]
Test Error: 
 Accuracy: 43.8%, Avg loss: 2.141258 

Epoch 2
-------------------------------
loss: 2.156942  [    0/60000]
loss: 2.147131  [ 6400/60000]
loss: 2.082487  [12800/60000]
loss: 2.088429  [19200/60000]
loss: 2.043563  [25600/60000]
loss: 1.977509  [32000/60000]
loss: 1.990417  [38400/60000]
loss: 1.918103  [44800/60000]
loss: 1.924612  [51200/60000]
loss: 1.835795  [57600/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.841659 

Epoch 3
-------------------------------
loss: 1.881821  [    0/60000]
loss: 1.849862  [ 6400/60000]
loss: 1.723581  [12800/60000]
loss: 1.757243  [19200/60000]
loss: 1.665043  [25600/60000]
loss: 1.611120  [32000/600